In [5]:
import os
import pickle as pkl
import pandas as pd
from IPython.display import display

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def load_processed_data(patient_id, data_dir='/Volumes/Seagate/pre_processed1/'):
    file_path = os.path.join(data_dir, f'{patient_id}.pkl')
    with open(file_path, 'rb') as f:
        data = pkl.load(f)
    return data

a = load_processed_data(11526147)
display(a)

,AlsUnitNo,TimeStamp,Label,Severity,SeverityColor,ABP_WAVEFORM,ECG_WAVEFORM,PPG_WAVEFORM,RESP_WAVEFORM,ABP_time_diff_sec,II_time_diff_sec,Pleth_time_diff_sec,Resp_time_diff_sec,SpO2_numeric,Pulse_numeric,ST_numeric,Tskin_numeric,ABP_numeric,NBP_numeric,HR_numeric,RR_numeric,SpO2_numeric_time_diff_sec,Pulse_numeric_time_diff_sec,ST_numeric_time_diff_sec,Tskin_numeric_time_diff_sec,ABP_numeric_time_diff_sec,NBP_numeric_time_diff_sec,HR_numeric_time_diff_sec,RR_numeric_time_diff_sec,NursingRecords_ba30,AdmissionIn,AdmissionOut,isView,isSelected,Classification,Comment
5,11526147,2024-08-09 17:01:02.143,[HR 108 >100],2,ShortYellow,"[-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50...","[-2.025, -2.03, -2.025, -2.04, -2.06, -2.06, -...","[0.48583984375, 0.51171875, 0.53515625, 0.5566...","[0.3463653023824068, 0.3420891875381796, 0.336...",20.48,0.0,0.0,0.0,100.0,84.0,NaN,NaN,NaN,NaN,NaN,19.0,0.320,0.320,43.200,0.320,19.777,5.143,0.320,0.320,"[{'시행일시': 2024-08-09 17:00:00, '간호진단프로토콜(코드명)'...",2024-08-09 16:57:50,2024-08-10 15:56:48,True,False,False,
6,11526147,2024-08-09 17:01:12.383,"[Cannot Analyze ST, Cannot Analyze QT, Cannot ...",5,SilentCyan,"[-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50...","[-0.765, -0.77, -0.765, -0.76, -0.755, -0.745,...","[0.84228515625, 0.84619140625, 0.84765625, 0.8...","[-0.11667684789248625, -0.13500305436774587, -...",10.24,0.0,0.0,0.0,99.7,83.0,NaN,NaN,NaN,NaN,NaN,21.0,0.320,0.320,32.960,0.320,9.537,15.383,0.320,0.320,"[{'시행일시': 2024-08-09 17:00:00, '간호진단프로토콜(코드명)'...",2024-08-09 16:57:50,2024-08-10 15:56:48,True,False,False,
7,11526147,2024-08-09 17:01:17.503,[ABP Zero+Check Cal],5,SilentCyan,"[-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50...","[-0.02, -0.02, -0.045, -0.045, -0.065, -0.07, ...","[0.41845703125, 0.40869140625, 0.39990234375, ...","[0.37935247403787414, 0.3952351863164325, 0.40...",5.12,0.0,0.0,0.0,99.4,83.0,NaN,NaN,NaN,NaN,101.0,16.0,0.320,0.320,27.840,0.320,4.417,20.503,0.320,0.320,"[{'시행일시': 2024-08-09 17:00:00, '간호진단프로토콜(코드명)'...",2024-08-09 16:57:50,2024-08-10 15:56:48,True,False,False,
8,11526147,2024-08-09 17:01:22.623,[Cannot Analyze ECG],4,Cyan,"[-50.0, -50.0, -50.0, -50.0, -50.0, -50.0, -50...","[-0.145, -0.155, -0.165, -0.17, -0.17, -0.175,...","[0.31689453125, 0.33935546875, 0.36669921875, ...","[-0.4276114844227245, -0.437996334758705, -0.4...",0.00,0.0,0.0,0.0,99.1,85.0,NaN,NaN,NaN,NaN,99.0,17.0,0.224,0.224,22.720,0.224,0.224,25.623,0.224,0.224,"[{'시행일시': 2024-08-09 17:00:00, '간호진단프로토콜(코드명)'...",2024-08-09 16:57:50,2024-08-10 15:56:48,True,False,False,
9,11526147,2024-08-09 17:01:37.983,"[Cannot Analyze ST, Cannot Analyze QT]",5,SilentCyan,"[58.3125, 56.875, 55.75, 55.8125, 55.4375, 52....","[-0.13, -0.115, -0.11, -0.11, -0.115, -0.115, ...","[0.4013671875, 0.390625, 0.38037109375, 0.3715...","[-0.26328649969456325, -0.2675626145387905, -0...",0.00,0.0,0.0,0.0,99.2,87.0,NaN,NaN,NaN,NaN,83.0,14.0,0.194,0.194,7.360,0.194,0.194,40.983,0.194,0.194,"[{'시행일시': 2024-08-09 17:00:00, '간호진단프로토콜(코드명)'...",2024-08-09 16:57:50,2024-08-10 15:56:48,True,False,False,
12,11526147,2024-08-09 17:01:53.343,"[ABP Zero+Check Cal, PPV bad ABP Signal]",5,SilentCyan,"[39.6875, 40.0, 42.9375, 45.25, 44.625, 40.187...","[0.12, 0.13, 0.12, 0.125, 0.115, 0.11, 0.12, 0...","[-0.08203125, -0.28369140625, -0.3916015625, -...","[-0.6212583995113011, -0.6291997556505803, -0....",0.00,0.0,0.0,0.0,99.9,81.0,0.0,NaN,NaN,NaN,83.0,15.0,0.194,0.194,0.194,0.194,0.194,56.343,0.194,0.194,"[{'시행일시': 2024-08-09 17:00:00, '간호진단프로토콜(코드명)'...",2024-08-09 16:57:50,2024-08-10 15:56:48,True,False,False,
